Group Name : Group 6

Student pace: Full time Hybrid

Instructor name:Diana Mongina


**BUSINESS PROBLEM**

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

**Objectives**

To find the genres that generates high income 

To find which movies performs best 

To determine whether movies highly rated generated higher revenue

To find the effect of production budget on profits ?revnue

To find if the release month  affects the rating



**Data Understanding**

In [1]:
# Importing libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
import sqlite3
from pandasql import sqldf

In [3]:
# SQLite connection.
conn=sqlite3.connect('./datasets/im.db')
query='''
SELECT *
    FROM sqlite_master;
'''
pd.read_sql(query,conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [4]:
# Joining of Tables
query2='''
SELECT *
    FROM movie_basics
    JOIN movie_ratings
    USING(movie_id);
'''
movie_basics_and_ratings=pd.read_sql(query2,conn)
movie_basics_and_ratings.dropna(inplace=True)
movie_basics_and_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65720 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         65720 non-null  object 
 1   primary_title    65720 non-null  object 
 2   original_title   65720 non-null  object 
 3   start_year       65720 non-null  int64  
 4   runtime_minutes  65720 non-null  float64
 5   genres           65720 non-null  object 
 6   averagerating    65720 non-null  float64
 7   numvotes         65720 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB


In [6]:
# Loading the movie budgets csv file.
movie_budgets = pd.read_csv('./datasets/tn.movie_budgets.csv', index_col=0)
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB


In [7]:
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [8]:
movie_budgets.worldwide_gross = movie_budgets.worldwide_gross.str.replace('[\$,]', '', regex=True).astype('int64')
movie_budgets.production_budget = movie_budgets.production_budget.str.replace('[\$,]', '', regex=True).astype('int64')
movie_budgets.domestic_gross = movie_budgets.domestic_gross.str.replace('[\$,]', '', regex=True).astype('int64')
movie_budgets.release_date = pd.to_datetime(movie_budgets.release_date)
print(movie_budgets.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 271.0+ KB
None


In [10]:
# Merging datasets.
pysqldf=lambda q: sqldf(q,globals())
query3='''
SELECT *
    FROM movie_basics_and_ratings AS mbr
    JOIN movie_budgets
    ON mbr.primary_title=movie_budgets.movie;
'''
movie_details_and_revenue=pysqldf(query3)
movie_details_and_revenue.release_date = pd.to_datetime(movie_details_and_revenue.release_date)
movie_details_and_revenue.info()

ImportError: Unable to find a usable engine; tried using: 'sqlalchemy'.
A suitable version of sqlalchemy is required for sql I/O support.
Trying to import the above resulted in these errors:
 - Pandas requires version '1.4.0' or newer of 'sqlalchemy' (version '1.3.19' currently installed).

In [11]:
# Top 10 genres ordered by domestic and worldwide gross.
top_10_genres=movie_details_and_revenue.groupby('genres')[['production_budget','domestic_gross','worldwide_gross']].mean().sort_values(by=['domestic_gross','worldwide_gross'],ascending=False).head(10)
top_10_genres

NameError: name 'movie_details_and_revenue' is not defined